DAR VISIBILIDADE DA PERFORMANCE DE NOVAS INDÚSTRIAS


In [ ]:
import streamlit as st
import pandas as pd
import sqlite3

# Caminho do banco
ARQUIVO_SQLITE = r"C:\Hbox\Banco de Dados\banco_local.db"

# Função para buscar cliente pelo CNPJ
def buscar_cliente(cnpj):
    conn = sqlite3.connect(ARQUIVO_SQLITE)
    query = "SELECT CODCLI, CLIENTE, CODUSUR1 AS RCA, CODUSUR2 AS RCA2, CGCENT FROM PCCLIENT WHERE CGCENT = ?"
    cliente = pd.read_sql(query, conn, params=(cnpj,))
    conn.close()
    return cliente


In [ ]:
import streamlit as st
import pandas as pd
import sqlite3
from datetime import date
from dateutil.relativedelta import relativedelta

# Caminho do banco
ARQUIVO_SQLITE = r"C:\Hbox\Banco de Dados\banco_local.db"

# ==========================
# Função para buscar cliente pelo CNPJ
# ==========================
def buscar_cliente(cnpj):
    cnpj = ''.join(filter(str.isdigit, str(cnpj)))
    conn = sqlite3.connect(ARQUIVO_SQLITE)
    query = """
        SELECT * FROM PCCLIENT 
        WHERE REPLACE(REPLACE(REPLACE(CGCENT,'.',''),'/',''),'-','') = ?
    """
    cliente = pd.read_sql(query, conn, params=(cnpj,))
    conn.close()
    return cliente

# ==========================
# Função para buscar pedidos do cliente
# ==========================
def buscar_pedidos(codcli, data_inicial=None, data_final=None, produto=None):
    conn = sqlite3.connect(ARQUIVO_SQLITE)
    params = [codcli]
    query = """
        SELECT m.CODPROD, p.DESCRICAO, m.QT, m.PUNIT, m.QT*m.PUNIT AS PRECO_FINAL, m.DTMOV
        FROM PCMOV m
        LEFT JOIN PCPRODUT p ON m.CODPROD = p.CODPROD
        WHERE m.CODCLI = ?
          AND (m.DTCANCEL IS NULL OR m.DTCANCEL = '')
    """

    if data_inicial and data_final:
        query += " AND DATE(m.DTMOV) BETWEEN ? AND ?"
        params.extend([data_inicial, data_final])
    
    if produto:
        query += " AND p.DESCRICAO LIKE ?"
        params.append(f"%{produto}%")
    
    query += " ORDER BY m.DTMOV DESC"
    pedidos = pd.read_sql(query, conn, params=params)
    conn.close()
    return pedidos

# ==========================
# Inicializa histórico do chat
# ==========================
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

if "cliente_temp" not in st.session_state:
    st.session_state.cliente_temp = None

st.title("💬 Chat Avançado de Cliente e Pedidos")

# ==========================
# Mostrar histórico do chat
# ==========================
for msg in st.session_state.chat_history:
    st.chat_message(msg["role"]).write(msg["content"])

# ==========================
# Input do usuário
# ==========================
user_input = st.chat_input("Digite o CNPJ do cliente ou comando (ex: 'pedidos mês anterior', 'produto: cerveja')...")

if user_input:
    st.session_state.chat_history.append({"role": "user", "content": user_input})
    resposta = ""

    # Comando: Buscar cliente por CNPJ
    if len(''.join(filter(str.isdigit, user_input))) >= 14:
        cnpj_digitado = ''.join(filter(str.isdigit, user_input))
        cliente = buscar_cliente(cnpj_digitado)
        if cliente.empty:
            resposta = "❌ Cliente não encontrado no cadastro."
        else:
            possible_name_cols = ['CLIENTE', 'NOME', 'NOME_CLIENTE']
            nome_col = next((c for c in possible_name_cols if c in cliente.columns), cliente.columns[0])
            possible_id_cols = ['CODCLI', 'CODCLIENTE', 'ID']
            id_col = next((c for c in possible_id_cols if c in cliente.columns), cliente.columns[0])
            
            st.session_state.cliente_temp = cliente.iloc[0][id_col]
            resposta = f"✅ Cliente encontrado: {cliente.iloc[0][nome_col]}"

            # Pedidos atuais
            pedidos = buscar_pedidos(st.session_state.cliente_temp)
            if pedidos.empty:
                resposta += "\n📌 Este cliente ainda não possui pedidos."
            else:
                resposta += f"\n📝 Pedidos atuais ({len(pedidos)} registros):"
                st.session_state.pedidos_temp = pedidos

    # Comando: Pedidos do mês anterior
    elif user_input.lower() in ["pedidos mês anterior", "pedidos mes anterior"]:
        if st.session_state.cliente_temp:
            hoje = date.today()
            primeiro_dia_mes = hoje.replace(day=1)
            data_inicial = (primeiro_dia_mes - relativedelta(months=1)).strftime("%Y-%m-%d")
            data_final = (primeiro_dia_mes - relativedelta(days=1)).strftime("%Y-%m-%d")
            pedidos = buscar_pedidos(st.session_state.cliente_temp, data_inicial, data_final)
            if pedidos.empty:
                resposta = "📌 Nenhum pedido no mês anterior."
            else:
                resposta = f"📝 Pedidos do mês anterior ({len(pedidos)} registros):"
                st.session_state.pedidos_temp = pedidos
        else:
            resposta = "Digite primeiro o CNPJ do cliente."

    # Comando: Filtrar por produto
    elif user_input.lower().startswith("produto:"):
        if st.session_state.cliente_temp:
            produto_nome = user_input.split(":",1)[1].strip()
            pedidos = buscar_pedidos(st.session_state.cliente_temp, produto=produto_nome)
            if pedidos.empty:
                resposta = f"📌 Nenhum pedido encontrado para o produto '{produto_nome}'."
            else:
                resposta = f"📝 Pedidos filtrados pelo produto '{produto_nome}' ({len(pedidos)} registros):"
                st.session_state.pedidos_temp = pedidos
        else:
            resposta = "Digite primeiro o CNPJ do cliente."

    else:
        resposta = "Comando não reconhecido. Use CNPJ, 'pedidos mês anterior' ou 'produto: <nome>'."

    # Exibir resposta do assistente
    st.session_state.chat_history.append({"role": "assistant", "content": resposta})
    st.chat_message("assistant").write(resposta)

    # Mostrar DataFrame de pedidos, se houver
    if "pedidos_temp" in st.session_state and not st.session_state.pedidos_temp.empty:
        st.dataframe(st.session_state.pedidos_temp)
        del st.session_state.pedidos_temp


In [6]:
import sqlite3
import pandas as pd
import os

# ==========================
# 🗄 Caminho do banco SQLite
# ==========================
ARQUIVO_SQLITE = r"C:\Hbox\Banco de Dados\banco_local.db"

# ==========================
# ⚙️ Tabelas e consultas SQL
# ==========================
TABELAS = {
    "PCMOV": """
        SELECT CODFILIAL, DTMOV, CODOPER, CODCLI, CODUSUR, CODPROD, 
               PUNIT, QT, CODFORNEC, NUMNOTA, NUMPED, CODPLPAG, DTCANCEL
        FROM PCMOV
    """,
    "PCUSUARI": "SELECT CODUSUR, NOME FROM PCUSUARI",
    "PCPRODUT": "SELECT CODPROD, DESCRICAO, CODFORNEC FROM PCPRODUT",
    "PCFORNEC": "SELECT CODFORNEC, FORNECEDOR FROM PCFORNEC",
    "PCCLIENT": """
        SELECT CODCLI, CGCENT, CLIENTE, CODUSUR1, CODUSUR2, BLOQUEIO, LIMCRED
        FROM PCCLIENT
    """
}

# ==========================
# 🔄 Função para carregar todas as tabelas
# ==========================
def carregar_tabelas_sqlite(caminho_banco: str):
    if not os.path.exists(caminho_banco):
        raise FileNotFoundError(f"Banco de dados não encontrado: {caminho_banco}")

    con = sqlite3.connect(caminho_banco)
    tabelas_dict = {}

    for nome, query in TABELAS.items():
        try:
            df = pd.read_sql(query, con)
            df.columns = df.columns.str.lower()  # converte todas as colunas para minúsculas
            tabelas_dict[nome.lower()] = df
            print(f"✅ Tabela {nome} carregada ({len(df)} registros).")
        except Exception as e:
            print(f"❌ Erro ao carregar tabela {nome}: {e}")

    con.close()
    return tabelas_dict


# ==========================
# ▶️ Execução de teste
# ==========================
if __name__ == "__main__":
    tabelas = carregar_tabelas_sqlite(ARQUIVO_SQLITE)

    # Exemplo de acesso aos DataFrames:
    df_pcmov = tabelas.get("pcmov")
    if df_pcmov is not None:
        print("\n📄 Exemplo PCMOV:")
        print(df_pcmov.head())


✅ Tabela PCMOV carregada (1263959 registros).
✅ Tabela PCUSUARI carregada (362 registros).
✅ Tabela PCPRODUT carregada (6170 registros).
✅ Tabela PCFORNEC carregada (1534 registros).
✅ Tabela PCCLIENT carregada (59388 registros).

📄 Exemplo PCMOV:
  codfilial                dtmov codoper   codcli  codusur  codprod  \
0         1  2020-10-26 00:00:00       E      NaN      NaN      644   
1         1  2020-10-26 00:00:00       S  18670.0    547.0      447   
2         1  2020-10-26 00:00:00       S  18670.0    547.0      644   
3         1  2020-10-26 00:00:00       E      NaN      NaN      447   
4         2  2021-01-22 00:00:00       E      NaN      0.0     2501   

       punit       qt  codfornec    numnota  numped  codplpag  \
0  25.427585  10080.0        483     9128.0     3.0       NaN   
1  23.500000   8064.0         78        1.0     4.0       1.0   
2  26.900000  10080.0         78        2.0     5.0       1.0   
3  21.230463   8064.0        483     9126.0     2.0       NaN   


In [ ]:

st.title("💬 Verificação de Cliente por CNPJ")

# Input do usuário
cnpj_input = st.text_input("Digite o CNPJ do cliente (somente números)")
cnpj = ''.join(filter(str.isdigit, cnpj_input))

if st.button("Verificar"):
    if cnpj_input:
        resultado = buscar_cliente(cnpj_input)
        if not resultado.empty:
            st.success(f"Cliente encontrado: {resultado.iloc[0]['CLIENTE']}")
            st.dataframe(resultado)
        else:
            st.error("Cliente não encontrado no cadastro.")
    else:
        st.warning("Digite um CNPJ válido.")


#ENVIAR OPORTUNIDADE DE CADASTRO POR RCA (INATIVO E/OU NOVOS CADASTROS)


#RELATORIO E VISÕES DE INDUSTRIAS.. PERFORMANCE, KPIs DAS INDUSTRIAS E DEMAIS VISÕES


#MANUTENÇÃO E GERENCIAMENTO DA PLATAFORMA BEES


#APURAÇÃO CAMPANHAS TIME OFF


#DAR VISIBILIDADE A BASE COMPRADORA DO CANAL: SEM COMPRA, INSTÁVEL E COMPRADOR


#FAZER CONTROLE DO CC (VERBAS APROVADAS PELA INDÚSTRIA)


#DAR VISIBILIDA AS MARGENS COMERCIAS (POR GERENTE E RCA)


#DAR VISIBILIDADE AS PERFORMANCES DE ENTREGA DO TIME LOGÍSTICO
